# Módulo 4: Interfaz de tasación inteligente (prototipo de producción)

**Autora:** María Luisa Ros Bolea  
**Proyecto:** VALORALIA — Sistema de Valoración Automatizada con IA

---

### Mi objetivo en este cuaderno

Aquí simulo el **despliegue en producción** del sistema Valoralia. Creo una función de tasación que:
1. Recibe los datos de un inmueble (metros, habitaciones, municipio...)
2. Opcionalmente, recibe una imagen del interior
3. Devuelve la valoración del modelo baseline y la del modelo híbrido
4. Aplica **reglas anti-alucinación** para evitar predicciones absurdas

También incluyo un simulador de escenarios macro (crisis, burbuja) y genero un informe de tasación exportable.


In [1]:
# ==============================================================================
# 1. CONFIGURACIÓN Y CARGA DE MODELOS
# ==============================================================================
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import pickle
import os, warnings
warnings.filterwarnings('ignore')

BASE_PATH = '/content/drive/MyDrive/TFM_Mejorado'
PROC_PATH = f'{BASE_PATH}/Data/Processed'
MODEL_PATH = f'{BASE_PATH}/Models'
IMG_PATH = f'{BASE_PATH}/Data/Images'
REPORT_PATH = f'{BASE_PATH}/Reports'

# Cargar modelos
with open(f'{MODEL_PATH}/modelo_baseline_rf.pkl', 'rb') as f:
    modelo_baseline = pickle.load(f)
with open(f'{MODEL_PATH}/modelo_hibrido_rf.pkl', 'rb') as f:
    modelo_hibrido = pickle.load(f)
with open(f'{PROC_PATH}/preprocessing_artifacts.pkl', 'rb') as f:
    artifacts = pickle.load(f)

scaler = artifacts['scaler']
le_municipio = artifacts['label_encoder_municipio']
FEATURES = artifacts['features_tabular']

print("✅ Modelos y artefactos cargados")
print(f"   Municipios disponibles: {list(le_municipio.classes_)}")


Mounted at /content/drive
✅ Modelos y artefactos cargados
   Municipios disponibles: ['Alcorcón', 'Fuenlabrada', 'Getafe', 'Leganés', 'Móstoles', 'Parla', 'Pinto']


## 2. Motor de tasación con lógica anti-alucinación

Esta función es el corazón del producto. Incorpora reglas de negocio que impiden que el modelo devuelva valores absurdos (por ejemplo, un piso de 30m² en Villaverde por 2 millones de euros).


In [2]:
# ==============================================================================
# 2. MOTOR DE TASACIÓN
# ==============================================================================

# Rangos de precio por m2 realistas (para validación anti-alucinación)
RANGOS_PRECIO_M2 = {
    'Madrid': (2500, 12000), 'Getafe': (1800, 4500), 'Alcorcón': (1800, 4200),
    'Leganés': (1500, 3800), 'Móstoles': (1500, 3500), 'Fuenlabrada': (1300, 3000),
    'Alcobendas': (2500, 6000), 'Pinto': (1500, 3500), 'Parla': (1200, 2800),
    'Torrejón de Ardoz': (1500, 3500), 'Alcalá de Henares': (1400, 3500),
    'San Sebastián de los Reyes': (2000, 5000), 'Pozuelo de Alarcón': (3000, 8000),
    'Las Rozas de Madrid': (2500, 6500), 'Majadahonda': (2500, 6500),
    'Rivas-Vaciamadrid': (2000, 5000), 'Coslada': (1800, 3800),
    'Tres Cantos': (2500, 5500), 'Valdemoro': (1500, 3500),
    'Collado Villalba': (1500, 3500), 'Aranjuez': (1000, 2800),
    'Arganda del Rey': (1300, 3000), 'Boadilla del Monte': (2500, 7000),
}
DEFAULT_RANGO = (1000, 8000)

def tasar_inmueble(tamano_m2, habitaciones, banos, municipio, estado,
                   planta='2', tiene_ascensor=True, tiene_terraza=False,
                   tiene_trastero=False, calefaccion='Central',
                   escenario='ESTABILIDAD', imagen_features=None):
    """
    Motor de tasación Valoralia.
    Devuelve valoración baseline + híbrida (si hay imagen) + validación.
    """
    # Codificación
    estado_map = {'A reformar': 0, 'Buen estado': 1, 'Obra nueva': 2}
    estado_cod = estado_map.get(estado, 1)

    def cod_planta(p):
        p = str(p).lower()
        if 'bajo' in p: return 0
        try: return min(int(''.join(filter(str.isdigit, p))), 10)
        except: return 2

    planta_cod = cod_planta(planta)

    try:
        municipio_cod = le_municipio.transform([municipio])[0]
    except:
        print(f"  ⚠️ Municipio '{municipio}' no reconocido, usando valor medio")
        municipio_cod = len(le_municipio.classes_) // 2

    calef_map = {c: i for i, c in enumerate(artifacts['label_encoder_calefaccion'].classes_)}
    calefaccion_cod = calef_map.get(calefaccion, 0)

    # Features
    ratio_hab_m2 = habitaciones / tamano_m2
    ratio_banos_hab = banos / max(habitaciones, 1)

    features_raw = np.array([[tamano_m2, habitaciones, banos, planta_cod,
                              int(tiene_ascensor), int(tiene_terraza), int(tiene_trastero),
                              calefaccion_cod, estado_cod, municipio_cod,
                              ratio_hab_m2, ratio_banos_hab]])

    features_scaled = scaler.transform(features_raw)

    # Predicción baseline
    precio_baseline = modelo_baseline.predict(features_scaled)[0]

    # Predicción híbrida (si hay features de imagen)
    precio_hibrido = None
    if imagen_features is not None:
        features_combined = np.concatenate([features_scaled.flatten(), imagen_features])
        precio_hibrido = modelo_hibrido.predict([features_combined])[0]

    # Escenario macro
    factores = {'ESTABILIDAD': 1.0, 'RECESIÓN': 0.90, 'CRASH': 0.80, 'BURBUJA': 1.15}
    factor = factores.get(escenario, 1.0)

    precio_baseline *= factor
    if precio_hibrido:
        precio_hibrido *= factor

    # Validación anti-alucinación
    rango = RANGOS_PRECIO_M2.get(municipio, DEFAULT_RANGO)
    precio_m2_pred = precio_baseline / tamano_m2
    alerta = None

    if precio_m2_pred < rango[0]:
        alerta = f"⚠️ Precio/m² ({precio_m2_pred:,.0f}€) por debajo del rango esperado para {municipio} ({rango[0]:,}€-{rango[1]:,}€)"
    elif precio_m2_pred > rango[1]:
        alerta = f"⚠️ Precio/m² ({precio_m2_pred:,.0f}€) por encima del rango esperado para {municipio} ({rango[0]:,}€-{rango[1]:,}€)"

    return {
        'precio_baseline': precio_baseline,
        'precio_hibrido': precio_hibrido,
        'precio_m2': precio_m2_pred,
        'escenario': escenario,
        'factor_macro': factor,
        'alerta': alerta
    }

print("✅ Motor de tasación cargado")


✅ Motor de tasación cargado


## 3. Simulación de tasaciones

Voy a probar el motor con varios escenarios realistas para demostrar que funciona correctamente.


In [3]:
# ==============================================================================
# 3. SIMULACIÓN DE TASACIONES
# ==============================================================================

casos_test = [
    {"nombre": "Piso lujo Pozuelo", "tamano_m2": 150, "habitaciones": 4, "banos": 3,
     "municipio": "Pozuelo de Alarcón", "estado": "Obra nueva", "tiene_ascensor": True, "tiene_terraza": True},
    {"nombre": "Piso estándar Leganés", "tamano_m2": 85, "habitaciones": 3, "banos": 1,
     "municipio": "Leganés", "estado": "Buen estado", "tiene_ascensor": True, "tiene_terraza": False},
    {"nombre": "Estudio Fuenlabrada", "tamano_m2": 45, "habitaciones": 1, "banos": 1,
     "municipio": "Fuenlabrada", "estado": "A reformar", "tiene_ascensor": False, "tiene_terraza": False},
    {"nombre": "Chalet Boadilla", "tamano_m2": 250, "habitaciones": 5, "banos": 3,
     "municipio": "Boadilla del Monte", "estado": "Obra nueva", "tiene_ascensor": False, "tiene_terraza": True},
]

print("=" * 70)
print("   SIMULACIÓN DE TASACIONES — VALORALIA SYSTEMS")
print("=" * 70)

for caso in casos_test:
    nombre = caso.pop('nombre')
    resultado = tasar_inmueble(**caso)

    print(f"\n🏠 {nombre}")
    print(f"   {caso['tamano_m2']}m² | {caso['habitaciones']} hab | {caso['municipio']} | {caso['estado']}")
    print(f"   💰 Valoración IA: {resultado['precio_baseline']:,.0f} € ({resultado['precio_m2']:,.0f} €/m²)")
    if resultado['alerta']:
        print(f"   {resultado['alerta']}")
    else:
        print(f"   ✅ Precio dentro del rango esperado")

    caso['nombre'] = nombre  # restaurar

# Simulación con escenarios macro
print(f"\n{'='*70}")
print("   TEST DE ESCENARIOS MACRO — Piso estándar Leganés (85m², 3 hab)")
print("="*70)

for esc in ['ESTABILIDAD', 'RECESIÓN', 'CRASH', 'BURBUJA']:
    r = tasar_inmueble(85, 3, 1, 'Leganés', 'Buen estado', escenario=esc)
    emoji = {'ESTABILIDAD': '🟢', 'RECESIÓN': '🟡', 'CRASH': '🔴', 'BURBUJA': '🟠'}
    print(f"   {emoji[esc]} {esc:<15s} → {r['precio_baseline']:>12,.0f} € (factor: x{r['factor_macro']})")


   SIMULACIÓN DE TASACIONES — VALORALIA SYSTEMS
  ⚠️ Municipio 'Pozuelo de Alarcón' no reconocido, usando valor medio

🏠 Piso lujo Pozuelo
   150m² | 4 hab | Pozuelo de Alarcón | Obra nueva
   💰 Valoración IA: 344,225 € (2,295 €/m²)
   ⚠️ Precio/m² (2,295€) por debajo del rango esperado para Pozuelo de Alarcón (3,000€-8,000€)

🏠 Piso estándar Leganés
   85m² | 3 hab | Leganés | Buen estado
   💰 Valoración IA: 204,145 € (2,402 €/m²)
   ✅ Precio dentro del rango esperado

🏠 Estudio Fuenlabrada
   45m² | 1 hab | Fuenlabrada | A reformar
   💰 Valoración IA: 100,202 € (2,227 €/m²)
   ✅ Precio dentro del rango esperado
  ⚠️ Municipio 'Boadilla del Monte' no reconocido, usando valor medio

🏠 Chalet Boadilla
   250m² | 5 hab | Boadilla del Monte | Obra nueva
   💰 Valoración IA: 387,668 € (1,551 €/m²)
   ⚠️ Precio/m² (1,551€) por debajo del rango esperado para Boadilla del Monte (2,500€-7,000€)

   TEST DE ESCENARIOS MACRO — Piso estándar Leganés (85m², 3 hab)
   🟢 ESTABILIDAD     →      204,14

In [4]:
# ==============================================================================
# 4. GENERACIÓN DE INFORME DE TASACIÓN (CSV)
# ==============================================================================

# Generar tasaciones masivas para el dataset de test
df = pd.read_csv(f'{PROC_PATH}/datos_procesados.csv')
df_sample = df.head(50)  # Muestra para el informe

registros = []
for _, row in df_sample.iterrows():
    r = tasar_inmueble(
        tamano_m2=row['tamano_m2'], habitaciones=row['habitaciones'],
        banos=row['banos'], municipio=row['municipio'], estado=row['estado'],
        planta=row['planta'], tiene_ascensor=bool(row['tiene_ascensor']),
        tiene_terraza=bool(row['tiene_terraza']),
        tiene_trastero=bool(row['tiene_trastero']),
        calefaccion=row['calefaccion']
    )
    registros.append({
        'id_anuncio': row['id_anuncio'],
        'municipio': row['municipio'],
        'tamano_m2': row['tamano_m2'],
        'precio_real': row['precio_actual'],
        'valoracion_ia': round(r['precio_baseline']),
        'precio_m2_ia': round(r['precio_m2']),
        'error_abs': abs(row['precio_actual'] - r['precio_baseline']),
        'alerta': r['alerta'] or 'OK'
    })

df_informe = pd.DataFrame(registros)
df_informe.to_csv(f'{REPORT_PATH}/informe_tasaciones.csv', index=False)

print(f"💾 Informe exportado: Reports/informe_tasaciones.csv ({len(df_informe)} tasaciones)")
print(f"\n📊 Estadísticas del informe:")
print(f"   Error absoluto medio: {df_informe['error_abs'].mean():,.0f}€")
print(f"   % alertas: {(df_informe['alerta']!='OK').mean()*100:.1f}%")
print(f"\n✅ Módulo 4 completado — Interfaz de tasación operativa")


💾 Informe exportado: Reports/informe_tasaciones.csv (50 tasaciones)

📊 Estadísticas del informe:
   Error absoluto medio: 9,001€
   % alertas: 0.0%

✅ Módulo 4 completado — Interfaz de tasación operativa
